In [1]:
import pandas as pd
import os 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import string
import matplotlib.pyplot as plt;

import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
import nltk
#for first time using nltk
# nltk.download ()

In [2]:
text_variable='clean_text'
target_feature='category'
splits=10
stop_words = stopwords.words('english') 
dataset_location="datasets/Twitter and Reddit Sentimental analysis Dataset/Twitter_Data.csv"

In [3]:
text_variable='clean_text'
target_feature='category'
dataset_location="datasets/Sentiment Analysis for Financial News/all-data.csv"

In [4]:
target_feature='airline_sentiment'
text_variable='text'
dataset_location='datasets/Twitter US Airline Sentiment/Tweets.csv'

In [5]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"(http\S+|http)", "", txt) # remove links 
    txt = re.sub('[^a-zA-Z ]+', '', txt) #only allows for letters
    txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stop_words ]) # stem & remove stop words
    return txt

def print_model_performance(target,predicted):
#     training_sample = model.predict(X_train)
#     testing_sample = model.predict(X_test)
#     print('training ')
#     print('train accuracy ',accuracy_score(training_sample, y_train))
#     print('\n testing  ')
#     print('test average accuracy ',accuracy_score(testing_sample, y_test))
#     print(confusion_matrix( y_test,testing_sample))
    print('outcome of training')
    print(classification_report( target,predicted))   #uncomment if you want to see full report 
    print('test average accuracy ',accuracy_score( target,predicted))
    print(confusion_matrix( target,predicted))


In [6]:
def sentiment_analyzer_scores(text, threshold=0.05, engl=True):
    analyser = SentimentIntensityAnalyzer()
    if engl:
        trans = text
    else:
        trans = translator.translate(text).text
    score = analyser.polarity_scores(trans)
    lb = score['compound']
    return lb
    if lb >= threshold:
        return 1
    elif (lb > -threshold) and (lb < threshold):
        return 0
    else:
        return -1
def train_test_split_features(train, test,train_feature, target_feature,vectorise):
    y_train = train[target_feature]   
    X_train = train[train_feature]
    y_test = test[target_feature]   
    X_test = test[train_feature]
    feature_names=[]
    if(vectorise):
        vect = TfidfVectorizer(min_df=5, ngram_range=(1, 4)) # create Count vectorizer.
        X_train = vect.fit(X_train).transform(X_train) # transform text_train  into a vector 
        X_test = vect.transform(X_test) 
        feature_names = vect.get_feature_names() # to return all words used in vectorizer
  
    return X_train, X_test, y_train, y_test, feature_names
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
#get this working a bit better later
def all_models():
    #Using the recomended classifiers
    #https://arxiv.org/abs/1708.05070
    GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
    RFC = RandomForestClassifier(n_estimators=500, max_features=0.25, criterion="entropy")
    SVM = SVC(C = 0.01, gamma=0.1, kernel="poly", degree=3, coef0=10.0)
    ETC = ExtraTreesClassifier(n_estimators=1000, max_features="log2", criterion="entropy")
    LR = LogisticRegression(C=1.5,fit_intercept=True)
    # Models that were not included in the paper not from SKlearn
    XGC = XGBClassifier()
    CBC = CatBoostClassifier(silent=True)
    light_gb = lgb.LGBMClassifier()
    models=[(LR, "linear_regression"),(ETC, "Extra_tree_classifier"),(SVM, "support_vector_classifier"), (RFC, "random_forest_classifier"), (GBC, "gradient_boosted_classifier"),
             (XGC, "XGBoost"),(light_gb,"Light_GBM"), (CBC, "catboost_classifier")]
    models=[(LR, "linear_regression"), (GBC, "gradient_boosted_classifier"),
             (XGC, "XGBoost"),(light_gb,"Light_GBM")]
#     models=[(LR, "linear_regression"),(SVM, "support_vector_classifier"), 
#               (CBC, "catboost_classifier"),(XGC, "XGBoost")]
#      (RFC, "random_forest_classifier"),(ETC, "Extra_tree_classifier"),(GBC, "gradient_boosted_classifier"),
#                (CBC, "catboost_classifier")]
    return models

In [7]:
def quick_run(df):
    train, test = train_test_split(df, test_size=0.99)
    train, validation = train_test_split(train, test_size=0.125)
    return train, validation, test

In [8]:

#data_df = pd.read_csv(dataset_location,encoding='ISO-8859-1',header='infer',)
#data_df.columns = [target_feature, text_variable]

In [9]:
data_df = pd.read_csv(dataset_location) 

data_df[text_variable] = data_df[text_variable].astype(str)
data_df[text_variable] = data_df[text_variable].apply(clean_text) 
data_df=data_df[[text_variable,target_feature]]
data_df = data_df.dropna()
# data_df,_,_=quick_run(data_df)
data_df=data_df.reset_index(drop=True)

data_df

,text,airline_sentiment
0,virginamerica dhepburn said,neutral
1,virginamerica plu youv ad commerci experi tacki,positive
2,virginamerica didnt today must mean need take ...,neutral
3,virginamerica realli aggress blast obnoxi ente...,negative
4,virginamerica realli big bad thing,negative
...,...,...
14635,americanair thank got differ flight chicago,positive
14636,americanair leav minut late flight warn commu...,negative
14637,americanair pleas bring american airlin blackb...,neutral
14638,americanair money chang flight dont answer pho...,negative


In [10]:

def run_features(df, model, splits,features=text_variable,vectorise=True, predict_probability=False):
    cv = KFold(n_splits=splits, random_state=42, shuffle=False)
    full_prediciton=[]
    for train_index, test_index in cv.split(df):
        train, test = df.loc[train_index], df.loc[test_index]

        X_train, X_test, y_train, y_test, feature_names=train_test_split_features(train,test,text_variable,target_feature, True)
#         model = LogisticRegressionCV()
        #to run faster
#         model = RandomForestClassifier()
        model.fit(X_train, y_train)
        #print_model_performance(lgstc, X_train, X_test, y_train, y_test)
        if (predict_probability==True):
            prediction = model.predict_proba(X_test)
        else:
            prediction = model.predict(X_test)
        full_prediciton.append(prediction)
    predictions=[]

    for set_of_prediction in full_prediciton:
        for predicted in set_of_prediction:
            predictions.append(predicted)
    return predictions
# to_dataset=data_df.copy()


model_predicted_names=[]
models=all_models()
df_copy=data_df.copy()
for model, name in models:
    print(name)
    import time
    start_time = time.time()
    predict_probability=True
    predictions=run_features(df_copy,model,splits,predict_probability=predict_probability)
    print("--- %s seconds ---" % (time.time() - start_time))
    predicted_name=name+'_prediction'
    negative_prob=[]
    neutral_prob=[]
    positive_prob=[]
    if (predict_probability):
        for neg, neut,pos in predictions:
            negative_prob.append(neg)
            neutral_prob.append(neut)
            positive_prob.append(pos)
        neg_predictions=predicted_name+'_neg'
        neut_predictions=predicted_name+'_neut'
        pos_predictions=predicted_name+'_pos'
        df_copy[neg_predictions]=negative_prob
        df_copy[neut_predictions]=neutral_prob
        df_copy[pos_predictions]=positive_prob

        model_predicted_names.append(neg_predictions)
        model_predicted_names.append(neut_predictions)
        model_predicted_names.append(pos_predictions)
        negative_prob.clear()
        neutral_prob.clear()
        positive_prob.clear()
    else:
#         df_copy[predicted_name]=predictions
#         model_predicted_names.append(predicted_name)
        df_copy[predicted_name]=predictions
        model_predicted_names.append(predicted_name)

        print_model_performance(df_copy[target_feature],predictions)
        predictions.clear()


linear_regression


/home/rick/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/home/rick/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/rick/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase 

--- 22.68628454208374 seconds ---
gradient_boosted_classifier
--- 52.52271509170532 seconds ---
XGBoost


/home/rick/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


--- 74.50928211212158 seconds ---
Light_GBM


/home/rick/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


--- 162.23888158798218 seconds ---


In [11]:
def sentiment_analyzer_scores(text, threshold=0.05, engl=True):
    analyser = SentimentIntensityAnalyzer()

    if engl:
        trans = text
    else:
        trans = translator.translate(text).text

    score = analyser.polarity_scores(trans)
    lb = score['compound']
    return lb
    if lb >= threshold:
        return 1
    elif (lb > -threshold) and (lb < threshold):
        return 0
    else:
        return -1

In [12]:
df_copy['polarity']=df_copy[text_variable].apply(lambda text: TextBlob(text).sentiment.polarity)
df_copy['subjectivity']=df_copy[text_variable].apply(lambda text: TextBlob(text).sentiment.subjectivity)
df_copy['vader_sentiment']=df_copy[text_variable].apply(lambda tweet: sentiment_analyzer_scores(tweet))
model_predicted_names.append('polarity')
model_predicted_names.append('subjectivity')
model_predicted_names.append('vader_sentiment')

In [13]:
df_copy

,text,airline_sentiment,linear_regression_prediction_neg,linear_regression_prediction_neut,linear_regression_prediction_pos,gradient_boosted_classifier_prediction_neg,gradient_boosted_classifier_prediction_neut,gradient_boosted_classifier_prediction_pos,XGBoost_prediction_neg,XGBoost_prediction_neut,XGBoost_prediction_pos,Light_GBM_prediction_neg,Light_GBM_prediction_neut,Light_GBM_prediction_pos,polarity,subjectivity,vader_sentiment
0,virginamerica dhepburn said,neutral,0.797402,0.163446,0.039152,0.765865,0.186492,0.047643,0.569338,0.348488,0.082174,0.658466,0.275765,0.065769,0.0000,0.000000,0.0000
1,virginamerica plu youv ad commerci experi tacki,positive,0.768217,0.103630,0.128153,0.859047,0.112271,0.028682,0.745079,0.164476,0.090446,0.717759,0.146752,0.135489,0.0000,0.000000,0.0000
2,virginamerica didnt today must mean need take ...,neutral,0.842602,0.143376,0.014022,0.685881,0.271451,0.042667,0.614816,0.365109,0.020074,0.614650,0.358835,0.026515,-0.3125,0.687500,0.0000
3,virginamerica realli aggress blast obnoxi ente...,negative,0.758093,0.132768,0.109139,0.842462,0.105130,0.052408,0.766549,0.120340,0.113111,0.752354,0.146953,0.100694,0.0000,0.000000,0.0000
4,virginamerica realli big bad thing,negative,0.844298,0.036344,0.119358,0.931683,0.045590,0.022727,0.840147,0.057549,0.102304,0.862668,0.043995,0.093338,-0.3500,0.383333,-0.5423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,americanair thank got differ flight chicago,positive,0.089353,0.520496,0.390151,0.030135,0.115817,0.854048,0.042256,0.153000,0.804744,0.047342,0.221495,0.731164,0.0000,0.000000,0.3612
14636,americanair leav minut late flight warn commu...,negative,0.990954,0.006345,0.002701,0.941381,0.038121,0.020498,0.976462,0.015679,0.007858,0.957470,0.030969,0.011561,-0.3000,0.600000,-0.1027
14637,americanair pleas bring american airlin blackb...,neutral,0.155839,0.627981,0.216181,0.440400,0.431016,0.128583,0.571948,0.277073,0.150979,0.333125,0.350045,0.316829,0.0000,0.000000,0.0000
14638,americanair money chang flight dont answer pho...,negative,0.900002,0.090315,0.009683,0.919845,0.063409,0.016746,0.915721,0.064107,0.020172,0.918684,0.072807,0.008508,0.0000,0.000000,0.2960


In [14]:
model = CatBoostClassifier(silent=True,task_type="CPU")
model =LogisticRegression()
predictions=run_features(df_copy,model,splits, features=model_predicted_names,vectorise=False)
print_model_performance(df_copy[target_feature],predictions)

/home/rick/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/home/rick/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/rick/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase 

outcome of training
              precision    recall  f1-score   support

    negative       0.80      0.93      0.86      9178
     neutral       0.66      0.46      0.54      3099
    positive       0.80      0.60      0.68      2363

    accuracy                           0.78     14640
   macro avg       0.75      0.66      0.70     14640
weighted avg       0.77      0.78      0.77     14640

test average accuracy  0.7793715846994536
[[8573  440  165]
 [1478 1422  199]
 [ 671  277 1415]]


In [15]:
print(model_predicted_names)

['linear_regression_prediction_neg', 'linear_regression_prediction_neut', 'linear_regression_prediction_pos', 'gradient_boosted_classifier_prediction_neg', 'gradient_boosted_classifier_prediction_neut', 'gradient_boosted_classifier_prediction_pos', 'XGBoost_prediction_neg', 'XGBoost_prediction_neut', 'XGBoost_prediction_pos', 'Light_GBM_prediction_neg', 'Light_GBM_prediction_neut', 'Light_GBM_prediction_pos', 'polarity', 'subjectivity', 'vader_sentiment']


In [16]:
df_copy

,text,airline_sentiment,linear_regression_prediction_neg,linear_regression_prediction_neut,linear_regression_prediction_pos,gradient_boosted_classifier_prediction_neg,gradient_boosted_classifier_prediction_neut,gradient_boosted_classifier_prediction_pos,XGBoost_prediction_neg,XGBoost_prediction_neut,XGBoost_prediction_pos,Light_GBM_prediction_neg,Light_GBM_prediction_neut,Light_GBM_prediction_pos,polarity,subjectivity,vader_sentiment
0,virginamerica dhepburn said,neutral,0.797402,0.163446,0.039152,0.765865,0.186492,0.047643,0.569338,0.348488,0.082174,0.658466,0.275765,0.065769,0.0000,0.000000,0.0000
1,virginamerica plu youv ad commerci experi tacki,positive,0.768217,0.103630,0.128153,0.859047,0.112271,0.028682,0.745079,0.164476,0.090446,0.717759,0.146752,0.135489,0.0000,0.000000,0.0000
2,virginamerica didnt today must mean need take ...,neutral,0.842602,0.143376,0.014022,0.685881,0.271451,0.042667,0.614816,0.365109,0.020074,0.614650,0.358835,0.026515,-0.3125,0.687500,0.0000
3,virginamerica realli aggress blast obnoxi ente...,negative,0.758093,0.132768,0.109139,0.842462,0.105130,0.052408,0.766549,0.120340,0.113111,0.752354,0.146953,0.100694,0.0000,0.000000,0.0000
4,virginamerica realli big bad thing,negative,0.844298,0.036344,0.119358,0.931683,0.045590,0.022727,0.840147,0.057549,0.102304,0.862668,0.043995,0.093338,-0.3500,0.383333,-0.5423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,americanair thank got differ flight chicago,positive,0.089353,0.520496,0.390151,0.030135,0.115817,0.854048,0.042256,0.153000,0.804744,0.047342,0.221495,0.731164,0.0000,0.000000,0.3612
14636,americanair leav minut late flight warn commu...,negative,0.990954,0.006345,0.002701,0.941381,0.038121,0.020498,0.976462,0.015679,0.007858,0.957470,0.030969,0.011561,-0.3000,0.600000,-0.1027
14637,americanair pleas bring american airlin blackb...,neutral,0.155839,0.627981,0.216181,0.440400,0.431016,0.128583,0.571948,0.277073,0.150979,0.333125,0.350045,0.316829,0.0000,0.000000,0.0000
14638,americanair money chang flight dont answer pho...,negative,0.900002,0.090315,0.009683,0.919845,0.063409,0.016746,0.915721,0.064107,0.020172,0.918684,0.072807,0.008508,0.0000,0.000000,0.2960
